### Create GIF

In [ ]:
import os
import json
import glob
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ================= HELPER FUNCTIONS ===================
# This function can preview a geometry data using matplot lib
def preview_geometries_without_title(geometry_data, plot_threshold=0.25, save_plot_as_png=False, path_to_save=None):
    # geometry_data must be an array of shape [32, 32, 64, 1]
    
    x = []
    y = []
    z = []

    data = np.squeeze(geometry_data, axis=-1)
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            for k in range(0, len(data[i][j])):
                if data[i][j][k] >= plot_threshold:
                    x.append(i)
                    y.append(j)
                    z.append(k)

    z_c = z

    # We decided to plot the results with a gradient color map so the depth and geometries are clearer to see. 
    from matplotlib.colors import ListedColormap

    N = 256
    vals = np.ones((N, 4))
    vals[:, 0] = np.linspace(1, 0, N)
    vals[:, 1] = np.linspace(43/N, 43/N, N)
    vals[:, 2] = np.linspace(82/N, 82/N, N)
    custom_cmp = ListedColormap(vals)

    fig = plt.figure(figsize=(5,5))
    fig = plt.axes(projection='3d')
    
    # Data for three-dimensional scattered points
    fig.scatter3D(np.array(x), np.array(y) * -1, np.array(z), cmap=custom_cmp, c=z_c)
    fig.set_zlim(0,64)
    fig.set_ylim(-64,0)
    fig.set_xlim(0,64)
    if (save_plot_as_png):
        plt.savefig(path_to_save + '.png')

In [ ]:
# Preview the generated samples, save them as pngs and create gif
def save_training_sequence_png(start_epoch, final_epoch, step, prediction_number, plot_threshold, directory_to_trained_samples, directory_to_save_gif):
    try:
        os.makedirs(directory_to_save_gif + "pngs/")
    except FileExistsError:
        print("The directory to save generated pngs already exists")
    for epoch in range(start_epoch, final_epoch, step):
        file_path = directory_to_trained_samples + 'epoch_' + str(epoch) + '_prediction_' + str(prediction_number) + '.json'
        with open(file_path) as json_file:
            dictionary = json.load(json_file)
            geometry = np.expand_dims(np.array(dictionary["geometry"]), axis=-1)
            preview_geometries_without_title(geometry, save_plot_as_png=True, path_to_save=directory_to_save_gif + "pngs/" + 'epoch_' + str(epoch) + '_prediction_' + str(prediction_number))
            
    anim_file_name = directory_to_save_gif + str(prediction_number) + '.gif'
    with imageio.get_writer(anim_file_name, mode='I', duration = 0.1) as writer:
        filenames = glob.glob(directory_to_save_gif + "pngs/" + '*.png')
        filenames = sorted(filenames)
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
folder_to_trained_samples = "./mogan_train/samples/"
folder_to_save_gif = "./mogan_train/gif/"

In [ ]:
save_training_sequence_png(0, 30, 1, 1, 0.2, folder_to_trained_samples, folder_to_save_gif)

### Find Similar Chairs from Dataset

In [ ]:
import json
import glob
import numpy as np

In [ ]:
# This function compares the l2 norm of the difference between the geometry and the geometries in the dataset

def find_similar_from_dataset_geometry(geometry, directory_to_json_dataset, num_of_results_to_return_in_ascending_order=5):

    # correctly format the geometry shape
    if np.array(geometry).shape == (1, 32, 32, 64, 1):
        geometry = np.squeeze(geometry, axis=0)
    if np.array(geometry).shape == (32, 32, 64, 1):
        geometry = np.squeeze(geometry, axis=-1)
    
    dataset_files = glob.glob(directory_to_json_dataset + "*.json")[:10]

    mse_values_list = []

    for ind, each_dataset_file in enumerate(dataset_files):
        
        with open(each_dataset_file) as json_file:
            dictionary = json.load(json_file)
        
        temp_mse = np.mean(np.square(np.array(geometry) - np.array(dictionary["geometry"])))
        
        mse_values_list.append((ind, temp_mse))
    
    dtype = [('file_path', int), ('mse', float)]
    ordered_mse_values_array = np.sort(np.array(mse_values_list, dtype=dtype), order='mse')
    
    result = []
    for ind, mse in ordered_mse_values_array[:num_of_results_to_return_in_ascending_order]:
        result.append((dataset_files[ind], mse))
  
    return result

In [ ]:
find_similar_from_dataset_geometry(np.random.rand(32,32,64), directory_to_json_dataset='./data/chair_voxel_dataset/')